In [ ]:
import librosa
import os

# Load the audio file
audio_file = '/Users/leek13/Desktop/2021_whole.wav'
y, sr = librosa.load(audio_file, sr=22050)

# Divide the audio into multiple one-hour segments
hour_len = 60*60*sr # length of one hour in samples
for i in range(0, len(y), hour_len):
    hour_audio = y[i:i+hour_len]
    hour_filename = f'hour_{i//hour_len}.wav'
    librosa.output.write_wav(hour_filename, hour_audio, sr)

    # Generate a spectrogram for each hour
    D = librosa.amplitude_to_db(librosa.stft(hour_audio), ref=np.max)
    fig, ax = plt.subplots()
    img = librosa.display.specshow(D, x_axis='time', y_axis='linear', ax=ax)
    fig.colorbar(img, ax=ax, format='%+2.0f dB')
    ax.set(title='Spectrogram')
#     plt.savefig(f'spectrogram_{i//hour_len}.png')
#     plt.close()
    plt.show()
    break


In [ ]:
print("test")

In [10]:
def generate_spectrogram(audio, sr=22050, n_fft=2048, hop_length=512):
    spectrogram = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length)
    log_spectrogram = librosa.amplitude_to_db(np.abs(spectrogram))
    return log_spectrogram

In [11]:
def find_infant_crying(log_spectrogram, threshold=-30, min_duration=1, sr=22050, hop_length=512):
    cry_intervals = []
    is_crying = False
    start_frame = 0

    for i, frame in enumerate(log_spectrogram.T):
        if np.mean(frame) > threshold:
            if not is_crying:
                start_frame = i
                is_crying = True
        else:
            if is_crying:
                duration = (i - start_frame) * hop_length / sr
                if duration >= min_duration:
                    start_time = start_frame * hop_length / sr
                    end_time = i * hop_length / sr
                    cry_intervals.append((start_time, end_time))
                is_crying = False

    return cry_intervals

In [ ]:
def divide_audio_into_hourly_segments(audio, sr=22050):
    seconds_per_hour = 3600
    samples_per_hour = seconds_per_hour * sr
    num_segments = int(np.ceil(len(audio) / samples_per_hour))

    segments = []
    for i in range(num_segments):
        start = i * samples_per_hour
        end = (i + 1) * samples_per_hour
        segment = audio[start:end]
        segments.append(segment)
    
    return segments

In [ ]:
 wav_file_path = '/Users/leek13/Desktop/2021_whole.wav'

# Load the audio file
audio = load_audio(wav_file_path)

# Generate the spectrogram
log_spectrogram = generate_spectrogram(audio)

# Find potential infant crying segments
cry_intervals = find_infant_crying(log_spectrogram)
print("test")

In [ ]:
# Print the results
if cry_intervals:
    print("Potential infant crying segments found:")
    for i, interval in enumerate(cry_intervals, start=1):
        start_time, end_time